<a href="https://colab.research.google.com/github/newalchemy/locality_sensitive_hashing/blob/main/FinalLSHCodeBasicAndGuidedRandomSampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initalization - pip install blocks

In [3]:
pip install kshingle

Install necessary modules and mount drive every session

> Add blockquote



In [4]:
pip install editdistance

In [5]:
pip install joblib

In [4]:
pip install psutil

# Class and parameter initalization

Initalize classes (must run first before any other code)

This is a copy of my github repository,

https://github.com/newalchemy/locality_sensitive_hashing

In [6]:
# -*- coding: utf-8 -*-
"""

Utils functions to be used in multiple experiments.


"""

import random;
import kshingle as ks;
from difflib import SequenceMatcher;
#from cdifflib import CSequenceMatcher;
import copy;
import math;
import editdistance;
import os.path
from os import path
import pickle
import json;
import joblib;
from joblib import Parallel, delayed
from functools import lru_cache
import gc
import sys
from google.colab import drive
import time;
import threading;
import concurrent.futures;
import psutil;

drive.mount('/content/drive')

def generate_random_dna_sequence(length, seed=-1):

    if (seed != -1):
        random.seed(seed);

    return_sequence_list = [];
    for i in range(0, length):
        val = random.randint(0,3);
        cur_char = int_to_dna_letter(val);
        return_sequence_list.append(cur_char);

    return_sequence = ''.join(return_sequence_list);

    return return_sequence;


def int_to_dna_letter(int_value):

    if int_value == 0:
        return 'A'
    elif int_value == 1:
        return 'T'
    elif int_value == 2:
        return 'G'
    elif int_value == 3:
        return 'C'
    else:
        raise Exception('Invalid value passed to int_to_dna_letter');

def dna_letter_to_int(dna_letter):

    if dna_letter == 'A':
        return 0;
    elif dna_letter == 'T':
        return 1
    elif dna_letter == 'G':
        return 2
    elif dna_letter == 'C':
        return 3
    else:
        raise Exception('Invalid value passed to dna_letter_to_int');


def change_2N_edit_distance_in_DNA_seq(dna_seq, num_changes_to_make_N):
    #Edit distance of sequence returned will be between N and 2N, tending towards 2N as N becomes smaller.
    #N = 5 -> ~90% probability of 2N
    #N = 10 -> 70% probability of 2N

    out_dna_seq = copy.copy(dna_seq);

    seq_length = len(dna_seq);

    for i in range(0, num_changes_to_make_N):
        #change_choice_int = random.randint(0,10);
        change_choice_int = 1;

        inx_char_to_change = random.randint(0, seq_length - 1);

        #Substitution - pick a random character to substitute
        if (change_choice_int == 0):
            try:
                old_char = dna_seq[inx_char_to_change];
            except IndexError:
                print('bad index: ', inx_char_to_change, '\n');
                print('len dna seq: ', len(dna_seq));

            old_char_int = dna_letter_to_int(old_char);

            new_char_int = random.randint(0, 3);
            while (new_char_int == old_char_int):
                new_char_int = random.randint(0, 3);
            new_char = int_to_dna_letter(new_char_int);
            out_dna_seq = out_dna_seq[:inx_char_to_change] + new_char + out_dna_seq[inx_char_to_change + 1:];

        #Insert/Delete - insert and delete a random character.
        else:
            new_char = int_to_dna_letter(random.randint(0,3));
            inx_to_delete = random.randint(0, seq_length);
            #If the distance between the two indicies is no more than 1 then you're performing substitution, not insert/delete swap
            while (abs(inx_to_delete - inx_char_to_change) <= 1):
                inx_to_delete = random.randint(0, seq_length);

            if (inx_to_delete < inx_char_to_change):
                out_dna_seq = out_dna_seq[:inx_to_delete] + out_dna_seq[inx_to_delete + 1:inx_char_to_change] + new_char + out_dna_seq[inx_char_to_change:seq_length];
            else:
                #deleting the last character is a special case
                if (inx_to_delete == seq_length):
                    out_dna_seq = out_dna_seq[:inx_char_to_change] + new_char + out_dna_seq[inx_char_to_change:seq_length - 1];
                else:
                    out_dna_seq = out_dna_seq[:inx_char_to_change] + new_char + out_dna_seq[inx_char_to_change:inx_to_delete] +  out_dna_seq[inx_to_delete + 1:seq_length];

    return out_dna_seq;



#There needs to be a more optimal way of doing this
def shingle_sequence(dna_sequence, shingle_size):
    ks_out = ks.shingleseqs_range(dna_sequence, n_min=shingle_size, n_max=shingle_size);
    out = tuple(ks_out[0]);
    return out;

def computeAllEditDistancesForQuery(query_dna_str, sample_id_to_sample_dict, num_dna_samples, max_edit_dist = -1):
    #Return all strings less than or equal to edit distance @param limit.  If Limit is -1, then return all.
    # Returns dictionary of sampleid : edit distance from query string mapping

    out_list = [];
    for i in range(0, num_dna_samples):
        my_sample_id = i;
        my_samp = sample_id_to_sample_dict[my_sample_id];

        dist = editdistance.eval(query_dna_str, my_samp);
        if (max_edit_dist != -1 and dist > max_edit_dist):
            continue;
        #dict_val = (my_sample_id, dist);
        #out_list.append(dict_val

        out_list.append((my_sample_id, dist));
    return out_list;

def subthread_computeAllEditDistancesForQuery(query_dna_str, sample_id_to_sample_dict, incstart, incend, max_edit_dist = -1):
    #Return all strings less than or equal to edit distance @param limit.  If Limit is -1, then return all.
    # Returns dictionary of sampleid : edit distance from query string mapping

    out_list = [];
    for i in range(incstart, incend):
        my_sample_id = i;
        my_samp = sample_id_to_sample_dict[my_sample_id];

        dist = editdistance.eval(query_dna_str, my_samp);
        if (max_edit_dist != -1 and dist > max_edit_dist):
            continue;
        #dict_val = (my_sample_id, dist);
        #out_list.append(dict_val

        out_list.append((my_sample_id, dist));
    return out_list;


def threaded_computeAllEditDistancesForQuery(query_dna_str_in, sample_id_to_sample_dict_in, num_dna_samples_in, max_edit_dist_in = -1):
    #Return all strings less than or equal to edit distance @param limit.  If Limit is -1, then return all.
    # Returns dictionary of sampleid : edit distance from query string mapping
    threads_count = int(psutil.cpu_count() / psutil.cpu_count(logical=False));
    inc_size = int(math.floor(num_dna_samples_in / threads_count));

    list_of_increment_tuples = [];
    for i in range(0, threads_count):
      inc_start = inc_size * i;
      inc_end = inc_size * (i+1);
      if (i == threads_count - 1):
        inc_end = num_dna_samples_in;
      list_of_increment_tuples.append((i, inc_start, inc_end));

    out_list = [];
    with concurrent.futures.ThreadPoolExecutor() as executor:
      futures = []
      for incremnt_tuple in list_of_increment_tuples:
        inc_start_inc = incremnt_tuple[1];
        inc_end_inc = incremnt_tuple[2];


        futures.append(
            executor.submit(
                subthread_computeAllEditDistancesForQuery, query_dna_str=query_dna_str_in, sample_id_to_sample_dict=sample_id_to_sample_dict_in, incstart=inc_start, incend=inc_end, max_edit_dist=max_edit_dist_in
            )
        )
      for future in concurrent.futures.as_completed(futures):
        out_list.extend(future.result());

    return out_list;



    out_list = [];
    for i in range(0, num_dna_samples):
        my_sample_id = i;
        my_samp = sample_id_to_sample_dict[my_sample_id];

        dist = editdistance.eval(query_dna_str, my_samp);
        if (max_edit_dist != -1 and dist > max_edit_dist):
            continue;
        #dict_val = (my_sample_id, dist);
        #out_list.append(dict_val

        out_list.append((my_sample_id, dist));
    return out_list;

def occurrences_of_shingle(sample, shingle):
    count = start = 0
    while True:
        start = sample.find(shingle, start) + 1
        if start > 0:
            count+=1
        else:
            return count

# REALLY inefficient but REALLY convinent
def find_longest_shingle_between_two_strings(string1, string2):
    # returns intersection shingles, number of intersecting shingles, size of shingles
    intersection = 1;
    shingle_size = 1;
    prev_intersection = 0;
    prev_intersect_size = 0;
    while (True):
        shingle_size = shingle_size + 1;
        ks_out1 = set(ks.shingleseqs_range(string1, n_min=shingle_size, n_max=shingle_size)[0]);
        ks_out2 = set(ks.shingleseqs_range(string2, n_min=shingle_size, n_max=shingle_size)[0]);
        intersect = ks_out1.intersection(ks_out2);
        intersection = len(intersect);
        if (intersection == 0):
            break;

        prev_intersection = intersect;
        prev_intersect_size = intersection;

    return prev_intersection, prev_intersect_size, shingle_size;
#test

#test_seq = generate_random_dna_sequence(200);
#print(test_seq);


#https://stackoverflow.com/questions/1456373/two-way-reverse-map

class TwoWayDict(dict):
    def __setitem__(self, key, value):
        # Remove any previous connections with these values
        if key in self:
            del self[key]
        if value in self:
            del self[value]
        dict.__setitem__(self, key, value)
        dict.__setitem__(self, value, key)

    def __delitem__(self, key):
        dict.__delitem__(self, self[key])
        dict.__delitem__(self, key)

    def __len__(self):
        """Returns the number of connections"""
        return dict.__len__(self) // 2



class DnaSequenceDataSampleAndLSHTable:


    def __init__(self, length_string, num_of_samples, shingle_size, sampling_choice, num_dna_seeds = -1, num_of_changes_per_seq = -1,  seed=-1):
        #Sampling choice 1 requires no additional params
        #Sampling choice 2 requires maximum_edit_distance, num_dna_seeds, num_changes_per_seq
        #Sampling choice 3 requires maximum_edit_distance
        self.length_string = length_string;
        self.num_of_samples = num_of_samples;
        self.seed = seed;
        self.shingle_size = shingle_size;
        self.num_random_dna_seeds = num_dna_seeds;
        self.sampling_choice = sampling_choice;
        self.max_num_of_changes_per_seq = num_of_changes_per_seq;

        self.sampleID_sample_map = TwoWayDict();
        self.shingleID_shingle_map = TwoWayDict();
        self.sampleID_to_shingleID_dict = {};
        #this is the LSH table.  Needs an initial entry to avoid key error.
        self.shingleID_to_sampleID_dict = {-1 : -1 };
        self.all_sample_ids = [];


        #Basic random sample
        if (sampling_choice == 1):
            self.__generateRandomSamples();
        #Guided random sample
        elif(sampling_choice == 2):
            self.__generateGuidedRandomSamples();
        #Centered random sample
        elif(sampling_choice == 3):
            self.__generateCenteredRandomSample();
        else:
            raise Exception("Bad sampling choice passed.  Sampling choice int:  ", sampling_choice, " valid values are 1,2,3")

        self.__shingleThisSampleAndBuildLSHTable();


                #print('created random samples, computing all pairs edit distance \n');

        #self.computeAllPairsEditDistance();

        #print('completed all pairs edit distance\n');

    def getLocalSamples(self, query_dna_sequence):
        #Given a query DNA sequence, return all 'local' samples based on the LSH scheme.
        shingles = shingle_sequence(query_dna_sequence, self.shingle_size);
        out_set = set();

        for i in range(0, len(shingles)):
            my_shingle = shingles[i];
            sample_list = self.getSamplesByShingle(my_shingle);

            if (sample_list != None):
                out_set.update(sample_list);

        out_samples = list(out_set);
        return out_samples;

    def getLocalSampleIds(self, query_dna_sequence):
        #Given a query DNA sequence, return all 'local' samples based on the LSH scheme.
        shingles = shingle_sequence(query_dna_sequence, self.shingle_size);
        out_set = set();

        for i in range(0, len(shingles)):
            my_shingle = shingles[i];
            sample_id_list = self.getSampleIdsByShingle(my_shingle);

            if (sample_id_list != None):
                out_set.update(sample_id_list);

        out_samples = list(out_set);
        return out_samples;


    def __generateRandomSamples(self):
        for i in range(0, self.num_of_samples):
            cur_sample = generate_random_dna_sequence(self.length_string, self.seed);
            self.sampleID_sample_map[i] = cur_sample;

    def __generateGuidedRandomSamples(self):
        iter_var = 0;

        #Put the seed DNA samples at the front of the dictionary
        #That's why we do this for loop twice
        for i in range(0, self.num_random_dna_seeds):
            cur_sample = generate_random_dna_sequence(self.length_string, self.seed);
            self.sampleID_sample_map[iter_var] = cur_sample;
            iter_var = iter_var + 1;

        self.num_replicas_per_seed = math.floor(self.num_of_samples/self.num_random_dna_seeds);
        for i in range(0, self.num_random_dna_seeds):
            cur_sample = self.getSamplebySampleID(i);
            for j in range(0, self.max_num_of_changes_per_seq):
                #Edit distance computation is very expensive, so we won't do it here.
                my_seq = change_2N_edit_distance_in_DNA_seq(cur_sample, self.max_num_of_changes_per_seq);
                self.sampleID_sample_map[iter_var] = my_seq;
                iter_var = iter_var + 1;

        #If there's any remaining due to the floor operation, just start populating
        while (iter_var < self.num_of_samples):
            samp_inx = random.randint(0, self.num_random_dna_seeds);
            cur_sample = self.getSamplebySampleID(samp_inx);
            my_seq = change_2N_edit_distance_in_DNA_seq(cur_sample, self.max_num_of_changes_per_seq);
            self.sampleID_sample_map[iter_var] = my_seq;
            iter_var = iter_var + 1;

    def __generateCenteredRandomSample(self):

        #        self.max_num_of_changes_per_seq = num_of_changes_per_seq;

        #Not implemented yet
        centered_sample = generate_random_dna_sequence(self.length_string, self.seed);
        self.sampleID_sample_map[0] = centered_sample;

        iter_var = 1;
        vals_each_distance = math.floor((self.num_of_samples - 1)/ self.max_num_of_changes_per_seq);

        for j in range(1, self.max_num_of_changes_per_seq):
            for r in range(1, vals_each_distance):
                my_seq = change_2N_edit_distance_in_DNA_seq(centered_sample, r);
                self.sampleID_sample_map[iter_var] = my_seq;
                iter_var = iter_var + 1;

        #Fill the rest with random distances
        while (iter_var < self.num_of_samples):
            my_dist = random.randint(1, self.max_num_of_changes_per_seq);
            my_seq = change_2N_edit_distance_in_DNA_seq(centered_sample, my_dist);
            self.sampleID_sample_map[iter_var] = my_seq;
            iter_var = iter_var + 1;

    def __shingleThisSampleAndBuildLSHTable(self):
        # Creates a dictionary of sample id - > [list of shingle ids]

        #For now, each sample can only have one shingling.  If the need arises this can be changed.

        #sample id to shingle id list

        itervar = 0;

        # O(M)
        for i in range(0, self.num_of_samples):
            my_sample = self.getSamplebySampleID(i);
            shingles = shingle_sequence(my_sample, self.shingle_size);

            shingle_id_for_doc_set = set();
            #O(N)
            for j in range(0, len(shingles)):
                my_shingle = shingles[j];

                #Create / Fetch unique shingle ID

                #If we've already seen the shingle, retrieve its id for the list
                shingle_id = self.getShingleIDbyShingle(my_shingle);

                if (shingle_id != -1):
                    shingle_id = self.getShingleIDbyShingle(my_shingle);
                    shingle_id_for_doc_set.add(shingle_id);

                #Otherwise, create a new id for it and add it to the known shingle list.
                else:
                    shingle_id = itervar;
                    self.shingleID_shingle_map[shingle_id] = my_shingle;
                    shingle_id_for_doc_set.add(shingle_id);

                    itervar = itervar + 1;


                # Now update the shingleID to sampleId dictionary
                k = self.shingleID_to_sampleID_dict.get(shingle_id);

                #Do we already have this shingle ID stored from a previous iteration?  If so, add the document to the dictionary - taking care not to duplicate the doc id
                if k:
                    ent = self.shingleID_to_sampleID_dict.get(shingle_id);
                    if (ent.count(i) == 0):
                        ent.append(i);
                        new_entry = [(shingle_id, ent)];
                        self.shingleID_to_sampleID_dict.update(new_entry);

                #otherwise create a new entry.
                else:
                    new_entry = [(shingle_id, [i])];
                    self.shingleID_to_sampleID_dict.update(new_entry);


            shingle_id_for_doc_list = list(shingle_id_for_doc_set);

            update = [(i, shingle_id_for_doc_list)];
            self.sampleID_to_shingleID_dict.update(update);
            self.shingleID_to_sampleID_dict.pop(-1, -1);


    def getSampleIdSampleMap(self):
        return self.sampleID_sample_map;

    def getOneWaySampleIdSampleMap(self):

        output = {};

        update_list = [];
        for i in range(0, self.num_of_samples):
            update_list.append([i, self.sampleID_sample_map[i]]);

        output.update(update_list);
        return output;

    def getTwoWaySampleIdSampleMap(self):
      return self.sampleID_to_shingleID_dict;

    # This is the number of initial samples used for guided random sampling
    def getNumDnaSeeds(self):
        return self.num_random_dna_seeds;

    def getMaxNumChangesPerSequence(self):
        return self.max_num_of_changes_per_seq;


    def getLengthDNAString(self):
        return self.length_string;

    def getNumOfDNASamples(self):
        return self.num_of_samples;

    def getSeed(self):
        return self.seed;

    def getAllShingleIds(self):
        return list(self.sampleID_to_shingleID_dict.keys());

    def getAllSampleIds(self):
        return list(self.sampleID_to_shingleID_dict.keys());

    def getSampleIDbySample(self, sample_str):
        return self.sampleID_sample_map[sample_str];

    def getSamplebySampleID(self, sample_id):

        return self.sampleID_sample_map[sample_id];

    def getShingleIDbyShingle(self, shingle_str):
      try:
        return self.shingleID_shingle_map[shingle_str];
      except KeyError:
        return -1;

    def getShinglebyShingleID(self, shingle_id):
        return self.shingleID_shingle_map[shingle_id];

    def getShingleIdsBySampleId(self, sample_id):
        return self.sampleID_to_shingleID_dict[sample_id];

    def getShinglesBySampleId(self, sample_id):
        shingle_id_list = self.sampleID_to_shingleID_dict[sample_id];
        shingles_list = [];
        for j in range(0, len(shingle_id_list)):
            my_shingle_id = shingle_id_list[j];
            my_shingle = self.shingleID_shingle_map[my_shingle_id];
            shingles_list.append(my_shingle);
        return shingles_list;

    def getShinglesBySample(self, sample_str):
        sample_id = self.getSampleIDbySample(sample_str);
        return self.getShinglesBySampleId(sample_id);


    def getShingleSize(self):
        return self.shingle_size;


    def getSamplesByShingle(self, shingle_str):
        shingle_id = self.getShingleIDbyShingle(shingle_str);

        if (shingle_id == -1):
            return None;
        return self.getSamplesByShingleId(shingle_id);

    def getSamplesByShingleId(self, shingle_id):
        sampleIds = self.getSampleIdsByShingleId(shingle_id);

        samples = [];

        if (sampleIds != None):
            for i in range(0, len(sampleIds)):
                my_sample_id = sampleIds[i];
                sample_str = self.getSamplebySampleID(my_sample_id);
                samples.append(sample_str);
        return samples;


    def getSampleIdsByShingleId(self, shingle_id):
        sampleIds = self.shingleID_to_sampleID_dict.get(shingle_id);
        return sampleIds;

    def getSampleIdsByShingle(self, shingle_str):
        shingle_id = self.getShingleIDbyShingle(shingle_str);

        if (shingle_id == -1):
            return None;
        return self.getSampleIdsByShingleId(shingle_id);




    def computeAllPairsEditDistance(self):
        #Brute force
        #Compute all pairs of an edit distance.

        self.all_pairs_dict = {};
        list_to_update = []
        for i in range(0, self.num_of_samples):
            s1 = self.sample_list[i];
            for j in range(i + 1, self.num_of_samples):
                s2 = self.sample_list[j];
                #only because I can't sort for some reason.  will make queries take longer.  fix later.
                key = s1.join(s2);
                val = editdistance.eval(s1, s2);
                list_to_update.append((key, val));

        self.all_pairs_dict.update([(key, val)]);









print('classes initalized\n');

#Define all Configurations:

#Params
length_of_dna_sequence = 1000; # N
num_of_dna_samples = 100000; # M
max_edit_distance = 10;

shingle_size = math.floor(length_of_dna_sequence/(max_edit_distance + 1));
sampling_choice = 1;

#These two are ignored if sampling choice = 1
num_dna_seeds = 500;
max_num_of_changes_per_seq = 5;

num_monte_carlos = 500;

sample_number = 5;

num_jobs = 10;
num_jobs_recall = 50;

num_changes = 5;


#End Params

if (sampling_choice == 2):
  num_monte_carlos = num_dna_seeds;


sample_num_str = str(sample_number);

basic_sample_path = '/content/drive/My Drive/colab/basicSampling/';
guided_sample_path = '/content/drive/My Drive/colab/guidedSampling/';
centered_sample_path = '/content/drive/My Drive/colab/centeredSampling/';

if path.exists(basic_sample_path) == False:
  os.mkdir(basic_sample_path)

if path.exists(guided_sample_path) == False:
  os.mkdir(guided_sample_path)

if path.exists(centered_sample_path) == False:
  os.mkdir(centered_sample_path)

choice_name = '';

if (sampling_choice == 1):
  choice_name = 'basic';
  path = basic_sample_path;
elif (sampling_choice == 2):
  choice_name = 'guided';
  path = guided_sample_path;
elif (sampling_choice == 3):
  choice_name = 'centered';
  path = centered_sample_path;
else:
  raise Exception("Bad sampling choice passed.  Sampling choice int:  ", sampling_choice, " valid values are 1,2,3");

sample_name = 'sample_' + choice_name + '_' + sample_num_str + '.pkl';
sample_id_to_sample_one_way_map_name = 'sample_' + choice_name + '_' + sample_num_str + '_sampleid_sample_one_way_map.pkl';
sample_id_to_sample_two_way_map_name = 'sample_' + choice_name + '_' + sample_num_str + '_sampleid_sample_two_way_map.pkl';


sample_path = path + sample_name;
sampleid_sample_oneway_path = path + sample_id_to_sample_one_way_map_name;
sampleid_sample_twoway_path = path + sample_id_to_sample_two_way_map_name;

mc_name_txt = 'monte_carlo_data_' + choice_name + '_' + sample_num_str + '_intermediate.txt';
mc_name_pkl = 'monte_carlo_data_' + choice_name + '_' + sample_num_str + '_intermediate.pkl';

mc_path_txt = path + mc_name_txt;
mc_path_pkl = path + mc_name_pkl;

exp_file_name_txt = 'full_monte_carlo_data' + choice_name + '_' + sample_num_str + '.txt';
exp_file_name_pkl = 'full_monte_carlo_data' + choice_name + '_' + sample_num_str + '.pkl';
exp_file_path_txt = path + exp_file_name_txt;
exp_file_path_pkl = path + exp_file_name_pkl;

precision_recall_file_txt = 'precision_recall_' + choice_name + '_' + sample_num_str + '.txt';
precision_recall_file_pkl = 'precision_recall_' + choice_name + '_' + sample_num_str + '.pkl';
precision_recall_file_path_txt = path + precision_recall_file_txt;
precision_recall_file_path_pkl = path + precision_recall_file_pkl;

print('paths initalized\n')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
classes initalized

paths initalized



#CPU maintance between runs

In [7]:
gc.collect();

try:
  del sample;
except:
  pass;
try:
  del sample_id_to_sample_one_way_map;
except:
  pass;
try:
  del sample_id_to_sample_two_way_map;
except:
  pass;

print('maintance finished\n')

maintance finished



# Data Generation Section - Basic and Guided Sampling
All in one block of code for effeciency

In [ ]:
## Generate and save Sample
startime = time.time();

print ('starting ', choice_name, ' random sample\n ')
print('Sample parameters:  length_of_dna_sequence:  ', length_of_dna_sequence, '\n');
print('num_of_dna_samples:  ', num_of_dna_samples, '\n');
print('shingle_size:  ', shingle_size, '\n');
print('num_dna_seeds:  ', num_dna_seeds, '\n');
print('max_num_of_changes_per_seq:  ', max_num_of_changes_per_seq, '\n');


sample = DnaSequenceDataSampleAndLSHTable(length_of_dna_sequence, num_of_dna_samples, shingle_size, sampling_choice, num_dna_seeds, max_num_of_changes_per_seq);
sample_id_to_sample_one_way_map = sample.getOneWaySampleIdSampleMap();
sample_id_to_sample_two_way_map = sample.getTwoWaySampleIdSampleMap();

print('sample generated \n');
endtime = time.time();
elapsed_secs = endtime - startime;
elapsed_mins = elapsed_secs / 60;
elapsed = str(elapsed_mins) + ' minutes';

'''
length_of_dna_sequence
num_of_dna_samples
shingle_size
'''

print('Time to generate sample: ', elapsed, '\n');
'''
with open(sample_path, 'wb') as f:
    pickle.dump(sample, f);
'''
with open(sampleid_sample_oneway_path, 'wb') as f:
    pickle.dump(sample_id_to_sample_one_way_map, f);
'''
with open(sampleid_sample_twoway_path, 'wb') as f:
    pickle.dump(sample_id_to_sample_two_way_map, f);
'''


## Generate random queries

output_data = {};

monte_carlo_data = {};

#Separate for loops is far more space effecient because we don't need sample object for the second loop.
#Second loop will be parallelized due to its call to O(N^2) edit distance function




def job_generateQueriesAndPerformLSHSearchInParallel_ForBasicSampling(i, query_sample_id):
  num_of_dna_samples = sample.getNumOfDNASamples();
  update_list = [];


  my_sample = sample.getSamplebySampleID(query_sample_id);



  this_query_dna = change_2N_edit_distance_in_DNA_seq(my_sample, max_num_of_changes_per_seq);

  #Perform LSH Search
  matching_dna_list = sample.getLocalSamples(this_query_dna);
  matching_dna_id_list = sample.getLocalSampleIds(this_query_dna);

  #Make output dictionary
  if (my_sample in matching_dna_list):
    foundOrigDna = "YES";
  else:
    foundOrigDna = "NO"


  update_list.append(('starting_sample_id', query_sample_id));
  update_list.append(('sampling_choice', 1)); # This is for basic random sampling only

  update_list.append(('query_for_mc', this_query_dna));
  update_list.append(('matching_dna_id_list', matching_dna_id_list));
  update_list.append(('number_of_matches', len(matching_dna_id_list)));
  update_list.append(('found_original_dna', foundOrigDna));

  output_dict = {};
  output_dict.update(update_list);
  return (i, output_dict);

def job_generateQueriesAndPerformLSHSearchInParallel_ForGuidedSampling(i):
  num_dna_seeds = sample.getNumDnaSeeds();
  update_list = [];


  this_query_dna = sample.getSamplebySampleID(i);

  #Perform LSH Search
  matching_dna_id_list = sample.getLocalSampleIds(this_query_dna);

  #Make output dictionary
  if (i in matching_dna_id_list):
    foundOrigDna = "YES";
  else:
    foundOrigDna = "NO"


  update_list.append(('starting_sample_id', i));
  update_list.append(('query_for_mc', 0)); # Kept for consistency across program
  update_list.append(('matching_dna_id_list', matching_dna_id_list));
  update_list.append(('number_of_matches', len(matching_dna_id_list)));
  update_list.append(('found_original_dna', foundOrigDna));

  output_dict = {};
  output_dict.update(update_list);
  return (i, output_dict);


if (sampling_choice == 1):

    query_sample_ids = random.sample(range(0, num_of_dna_samples), num_monte_carlos);

    output = Parallel(n_jobs=num_jobs, prefer='threads', verbose=60 )(delayed(job_generateQueriesAndPerformLSHSearchInParallel_ForBasicSampling)(i, query_sample_ids[i]) for i in range(0,num_monte_carlos));
elif(sampling_choice == 2):
    output = Parallel(n_jobs=num_jobs, prefer='threads', verbose=60)(delayed(job_generateQueriesAndPerformLSHSearchInParallel_ForGuidedSampling)(i) for i in range(0,num_dna_seeds));
else:
  raise Exception("Bad sampling choice passed.  Sampling choice int:  ", sampling_choice, " valid values are 1,2.  For 3 use other block");

output_dict = {};
output_dict.update(output);

print(output_dict);

print('queries generationed \n');

#We only need the 1 way sampleid -> sample map from this point on



#Delete sample because it takes a lot of space
del sample;

def job_getEditDistanceForAllLSHMatches(i, my_mc_data):
  out_this_mc = {};

  sampling_choice = my_mc_data['sampling_choice'];
  starting_sample_id = my_mc_data['starting_sample_id'];
  if (sampling_choice == 1):
    query_for_mc = my_mc_data['query_for_mc'];
  else:
    query_for_mc = sample_id_to_sample_one_way_map[starting_sample_id];
  matching_dna_id_list = my_mc_data['matching_dna_id_list'];
  num_matches = my_mc_data['number_of_matches'];

  matchid_editdistance_pairs = [];
  for j in range(0, num_matches):
    cur_match_id = matching_dna_id_list[j];
    cur_match = sample_id_to_sample_one_way_map[cur_match_id];
    cur_edit_distance = editdistance.eval(query_for_mc, cur_match);

    entry = (cur_match_id, cur_edit_distance);
    matchid_editdistance_pairs.append(entry);

  my_mc_out = {};
  my_mc_out['starting_sample_id'] = starting_sample_id;
  my_mc_out['sampling_choice'] = 2; #This is for guided random sampling only

  if (sampling_choice == 1):
    my_mc_out['query_for_mc'] = query_for_mc
  else:
    my_mc_out['query_for_mc'] = 0; # Kept for consistency across program

  my_mc_out['number_of_matches'] = num_matches
  my_mc_out['matchid_editdistance_pairs'] = matchid_editdistance_pairs

  return (i , my_mc_out);

# for sampling_choice == 2, num_monte_carls = num_dna_seeds
if (sampling_choice == 1):
  print(output_dict[0]);
  output = Parallel(n_jobs=num_jobs, prefer='threads')(delayed(job_getEditDistanceForAllLSHMatches)(i, output_dict[i]) for i in range(0, num_monte_carlos));
else:
  output = Parallel(n_jobs=num_jobs, prefer='threads', verbose=60)(delayed(job_getEditDistanceForAllLSHMatches)(i, output_dict[i]) for i in range(0, num_dna_seeds));

my_mc_data_updated = {};

my_mc_data_updated.update(output);


with open(mc_path_pkl, 'wb') as f:
    pickle.dump(my_mc_data_updated, f);

with open(mc_path_txt, 'w') as convert_file:
     convert_file.write(json.dumps(my_mc_data_updated))

print('test 1 finished successfully! \n');

def job_getRecallForAllQueryStrings(i, cur_mc, sample_id_to_sample_one_way_map_in, num_of_dna_samples):
  out_list_for_mc = [];
  starting_sample_id_int = cur_mc['starting_sample_id'];
  sampling_choice = cur_mc['sampling_choice'];

  if (sampling_choice == 1):
    query_for_mc_str = cur_mc['query_for_mc'];
  else:
    query_for_mc_str = sample_id_to_sample_one_way_map_in[starting_sample_id_int];

  num_matches_int = cur_mc['number_of_matches'];
  matched_sample_ids_edit_distance_list_of_tuples = cur_mc['matchid_editdistance_pairs'];

  list_of_sample_id_edit_distance_pairs = computeAllEditDistancesForQuery(query_for_mc_str, sample_id_to_sample_one_way_map_in, num_of_dna_samples, max_edit_distance);

  del sample_id_to_sample_one_way_map_in;
  gc.collect();

  num_lsh_correctly_matched = 0;
  num_lsh_wrongly_matched = 0;

  num_actual_correctly_matched = 0;

  for j in range(len(matched_sample_ids_edit_distance_list_of_tuples)):
    cur_tuple = matched_sample_ids_edit_distance_list_of_tuples[j];
    cur_sample_id = cur_tuple[0];
    cur_edit_distance = cur_tuple[1];

    if (cur_edit_distance <= max_edit_distance):
      num_lsh_correctly_matched = num_lsh_correctly_matched + 1;
    else:
      num_lsh_wrongly_matched = num_lsh_wrongly_matched + 1;

  for k in range(0, len(list_of_sample_id_edit_distance_pairs)):
    cur_pair = list_of_sample_id_edit_distance_pairs[k];
    cur_pair_sample_id = cur_pair[0];
    cur_pair_edit_distance = cur_pair[1];
    if (cur_pair_edit_distance <= max_edit_distance):
      num_actual_correctly_matched = num_actual_correctly_matched + 1;

  precision = num_lsh_correctly_matched / (num_lsh_correctly_matched + num_lsh_wrongly_matched);
  recall = num_lsh_correctly_matched / num_actual_correctly_matched;
  tup1 = ('precision', precision);
  tup2 = ('recall', recall);
  tup3 = ('starting_sample_id', starting_sample_id_int);
  tup4 = ('query_for_mc', query_for_mc_str);
  tup5 = ('num_matches', num_matches_int);
  out_list_for_mc.append(tup1);
  out_list_for_mc.append(tup2);
  out_list_for_mc.append(tup3);
  out_list_for_mc.append(tup4);
  out_list_for_mc.append(tup5);

  recall_dict_out = {};
  recall_dict_out.update(out_list_for_mc);
  return (i, recall_dict_out);


output = Parallel(n_jobs=num_jobs_recall, verbose=60)(delayed(job_getRecallForAllQueryStrings)(i, my_mc_data_updated[i], sample_id_to_sample_one_way_map, num_of_dna_samples) for i in range(0, num_monte_carlos));

output_dict = {};
output_dict.update(output);

with open(precision_recall_file_path_pkl, 'wb') as f:
    pickle.dump(output_dict, f);

with open(precision_recall_file_path_txt, 'w') as convert_file:
     convert_file.write(json.dumps(output_dict))
print('entire thing finished!')


#Recall Only

Time intensive code, so a replica of it is placed in its own section

In [ ]:

with open(mc_path_pkl, 'rb') as f:
    my_mc_data_updated = pickle.load(f);

with open(sampleid_sample_oneway_path, 'rb') as f:
    sample_id_to_sample_one_way_map = pickle.load(f);



def job_getRecallForAllQueryStrings(i, cur_mc, sample_id_to_sample_one_way_map_in, num_of_dna_samples):
  out_list_for_mc = [];
  starting_sample_id_int = cur_mc['starting_sample_id'];
  sampling_choice = cur_mc['sampling_choice'];

  if (sampling_choice == 1):
    query_for_mc_str = cur_mc['query_for_mc'];
  else:
    query_for_mc_str = sample_id_to_sample_one_way_map_in[starting_sample_id_int];

  num_matches_int = cur_mc['number_of_matches'];
  matched_sample_ids_edit_distance_list_of_tuples = cur_mc['matchid_editdistance_pairs'];

  list_of_sample_id_edit_distance_pairs = computeAllEditDistancesForQuery(query_for_mc_str, sample_id_to_sample_one_way_map_in, num_of_dna_samples, max_edit_distance);

  del sample_id_to_sample_one_way_map_in;
  gc.collect();

  num_lsh_correctly_matched = 0;
  num_lsh_wrongly_matched = 0;

  num_actual_correctly_matched = 0;

  for j in range(len(matched_sample_ids_edit_distance_list_of_tuples)):
    cur_tuple = matched_sample_ids_edit_distance_list_of_tuples[j];
    cur_sample_id = cur_tuple[0];
    cur_edit_distance = cur_tuple[1];

    if (cur_edit_distance <= max_edit_distance):
      num_lsh_correctly_matched = num_lsh_correctly_matched + 1;
    else:
      num_lsh_wrongly_matched = num_lsh_wrongly_matched + 1;

  for k in range(0, len(list_of_sample_id_edit_distance_pairs)):
    cur_pair = list_of_sample_id_edit_distance_pairs[k];
    cur_pair_sample_id = cur_pair[0];
    cur_pair_edit_distance = cur_pair[1];
    if (cur_pair_edit_distance <= max_edit_distance):
      num_actual_correctly_matched = num_actual_correctly_matched + 1;

  precision = num_lsh_correctly_matched / (num_lsh_correctly_matched + num_lsh_wrongly_matched);
  recall = num_lsh_correctly_matched / num_actual_correctly_matched;
  tup1 = ('precision', precision);
  tup2 = ('recall', recall);
  tup3 = ('starting_sample_id', starting_sample_id_int);
  tup4 = ('query_for_mc', query_for_mc_str);
  tup5 = ('num_matches', num_matches_int);
  out_list_for_mc.append(tup1);
  out_list_for_mc.append(tup2);
  out_list_for_mc.append(tup3);
  out_list_for_mc.append(tup4);
  out_list_for_mc.append(tup5);

  recall_dict_out = {};
  recall_dict_out.update(out_list_for_mc);
  return (i, recall_dict_out);


output = Parallel(n_jobs=num_jobs_recall, verbose=60)(delayed(job_getRecallForAllQueryStrings)(i, my_mc_data_updated[i], sample_id_to_sample_one_way_map, num_of_dna_samples) for i in range(0, num_monte_carlos));

output_dict = {};
output_dict.update(output);

with open(precision_recall_file_path_pkl, 'wb') as f:
    pickle.dump(output_dict, f);

with open(precision_recall_file_path_txt, 'w') as convert_file:
     convert_file.write(json.dumps(output_dict))
print('entire thing finished!')


# Data Generation Section - Centered Random Sampling




In [8]:
#Params
length_of_dna_sequence = 1000; # = N
num_of_dna_samples = 50001;# = M
max_edit_distance = 50; # Parameter R from slides
shingle_size = math.floor(length_of_dna_sequence/(max_edit_distance + 1)); #S
#shingle_size = 47; # = floor(N/(R+1))

#These two are ignored if sampling choice = 1
#num_dna_seeds = 500; # only for choice=2
num_of_changes_per_seq = 300;
#max num of changes per sequence for sampling choice

num_monte_carlos = 100;

num_of_experiments = 10;

start_sample_number = 2;

num_jobs = 10;
num_jobs_recall = 50;


max_edit_distance = 20; # Parameter R from slides


#End Params

sampling_choice = 3;
choice_name = 'centered';
centered_sample_path = '/content/drive/My Drive/colab/centeredSampling/';

if (start_sample_number >= num_of_experiments):
  raise Exception('parameter conflict, start_sample_number >= num_of_experiments \n');


def job_computePrecisionRecallForCenterOfSample(i, cur_sample):
  dist = editdistance.eval(center_of_sample, cur_sample);

  is_found_by_LSH_cur_sample = i in lsh_matched_ids_set;
  is_match_cur_sample = dist <= max_edit_distance;

  samp_dict = {};
  #samp_dict['sample_id'] = i;
  samp_dict['edit_distance'] = dist;
  samp_dict['is_found_by_LSH'] = is_found_by_LSH_cur_sample;
  samp_dict['is_match'] = is_match_cur_sample;


  return (i, samp_dict);


for s in range(start_sample_number, num_of_experiments):

  sample_number = s + 1; #Keep naming convention consistent across datasets
  sample_num_str = str(sample_number);

  sample_id_to_sample_one_way_map_name = 'sample_' + choice_name + '_' + sample_num_str + '_sampleid_sample_one_way_map.pkl';
  sampleid_sample_oneway_path = centered_sample_path + sample_id_to_sample_one_way_map_name;

  edit_distance_data_txt = centered_sample_path + 'edit_distance_to_precision_recall_data_' + sample_num_str + '.txt';
  edit_distance_data_pkl = centered_sample_path + 'edit_distance_to_precision_recall_data_' + sample_num_str + '.pkl';

  centered_file_precision_recall_data_txt = centered_sample_path + 'center_precision_recall_data_' + sample_num_str + '.txt';
  centered_file_precision_recall_data_pkl = centered_sample_path + 'center_precision_recall_data_' + sample_num_str + '.pkl';

  matched_key_str = 'num_lsh_matched';
  not_matched_key_str = 'num_lsh_didnt_match';


  ## Generate and save Sample
  startime = time.time();

  print('starting ', choice_name, ' random sample\n ')
  print('Sample parameters:  length_of_dna_sequence:  ', length_of_dna_sequence, '\n');
  print('num_of_dna_samples:  ', num_of_dna_samples, '\n');
  print('shingle_size:  ', shingle_size, '\n');
  print('num_dna_seeds:  ', num_dna_seeds, '\n');
  print('num_of_changes_per_seq:  ', num_of_changes_per_seq, '\n');


  sample = DnaSequenceDataSampleAndLSHTable(length_of_dna_sequence, num_of_dna_samples, shingle_size, sampling_choice, num_dna_seeds, num_of_changes_per_seq);
  sample_id_to_sample_one_way_map = sample.getOneWaySampleIdSampleMap();
  sample_id_to_sample_two_way_map = sample.getTwoWaySampleIdSampleMap();

  print('sample generated \n');
  endtime = time.time();
  elapsed_secs = endtime - startime;
  elapsed_mins = elapsed_secs / 60;
  elapsed = str(elapsed_mins) + ' minutes';


  print('Time to generate sample: ', elapsed, '\n');
  '''
  with open(sample_path, 'wb') as f:
    pickle.dump(sample, f);
  '''
  with open(sampleid_sample_oneway_path, 'wb') as f:
    pickle.dump(sample_id_to_sample_one_way_map, f);
  '''
  with open(sampleid_sample_twoway_path, 'wb') as f:
    pickle.dump(sample_id_to_sample_two_way_map, f);
  '''


  center_of_sample = sample.getSamplebySampleID(0);

  lsh_matches_ids = sample.getLocalSampleIds(center_of_sample);

  del sample;

  num_lsh_correctly_matched = 0;
  num_lsh_wrongly_matched = 0;

  num_actual_matched = 0;

  lsh_matched_ids_set = set();
  lsh_matched_ids_set.update(lsh_matches_ids);


  output = Parallel(n_jobs=num_jobs_recall, verbose=60)(delayed(job_computePrecisionRecallForCenterOfSample)(i, sample_id_to_sample_one_way_map[i]) for i in range(0, num_of_dna_samples));

  par_output_dict = {};
  #sample id -> edit_distance, is_found_by_lsh, is_match
  par_output_dict.update(output);

  num_lsh_correctly_matched = 0;
  num_lsh_wrongly_matched = 0;

  num_actual_matched = 0;

  master_edit_distance_dict = {};


  for i in range(0, num_of_dna_samples):
    my_samp_dict = par_output_dict[i];

    is_found_by_lsh = my_samp_dict['is_found_by_LSH'];
    is_match = my_samp_dict['is_match'];
    my_edit_distance = my_samp_dict['edit_distance'];

    if (is_found_by_lsh and is_match):
      num_lsh_correctly_matched = num_lsh_correctly_matched + 1;
      num_actual_matched = num_actual_matched + 1;
    elif (is_found_by_lsh and not is_match):
      num_lsh_wrongly_matched = num_lsh_wrongly_matched + 1;
    elif (not is_found_by_lsh and is_match):
      num_actual_matched = num_actual_matched + 1;
    else:
      #not is_found_by_lsh and is_match
      pass;



    k = master_edit_distance_dict.get(my_edit_distance);

    if (k == None):
      iter_edit_distance_dict = {};

      if (is_found_by_lsh):
        iter_edit_distance_dict[matched_key_str] = 1;
      else:
        iter_edit_distance_dict[not_matched_key_str] = 1;
        #not is_found_by_lsh and is_match
        pass;
      master_edit_distance_dict[my_edit_distance] = iter_edit_distance_dict;

      iter_edit_distance_dict['count_edit_distance'] = 1;

    else:

      iter_edit_distance_dict = k;

      if (is_found_by_lsh):
        if (matched_key_str in iter_edit_distance_dict.keys()):
          iter_edit_distance_dict[matched_key_str] = iter_edit_distance_dict[matched_key_str] + 1;
        else:
          iter_edit_distance_dict[matched_key_str] = 1;
      else:
        if (not_matched_key_str in iter_edit_distance_dict.keys()):
          iter_edit_distance_dict[not_matched_key_str] = iter_edit_distance_dict[not_matched_key_str] + 1;
        else:
          iter_edit_distance_dict[not_matched_key_str] = 1;
      master_edit_distance_dict[my_edit_distance] = iter_edit_distance_dict;

      iter_edit_distance_dict['count_edit_distance'] = iter_edit_distance_dict['count_edit_distance'] + 1;



  for key in master_edit_distance_dict:
    cur_dict = master_edit_distance_dict[key];

    if (matched_key_str in cur_dict.keys()):
      num_lsh_matched = cur_dict[matched_key_str];
    else:
      iter_edit_distance_dict[matched_key_str] = 0;
      num_lsh_matched = 0;

    if (not_matched_key_str in cur_dict.keys()):
      num_lsh_didnt_match = cur_dict[not_matched_key_str];
    else:
      iter_edit_distance_dict[not_matched_key_str] = 0;
      num_lsh_didnt_match = 0;

    total_at_edit_distance = num_lsh_matched + num_lsh_didnt_match;
    if (total_at_edit_distance == 0):
      master_edit_distance_dict[key]['percent_LSH_returned_at_this_edit_distance'] = -1;
    else:
      master_edit_distance_dict[key]['percent_LSH_returned_at_this_edit_distance'] = (num_lsh_matched / (total_at_edit_distance))*100;



  precision = num_lsh_correctly_matched / (num_lsh_correctly_matched + num_lsh_wrongly_matched);
  recall = num_lsh_correctly_matched / num_actual_matched;

  output_dict1 = {};

  output = [];
  output.append(('sample_number', sample_number));
  output.append(('precision', precision));
  output.append(('recall', recall));
  output_dict1.update(output);

  with open(centered_file_precision_recall_data_pkl, 'wb') as f:
    pickle.dump(output_dict1, f);

  with open(centered_file_precision_recall_data_txt, 'w') as convert_file:
    convert_file.write(json.dumps(output_dict1))

  with open(edit_distance_data_pkl, 'wb') as f:
    pickle.dump(master_edit_distance_dict, f);

  with open(edit_distance_data_txt, 'w') as convert_file:
    convert_file.write(json.dumps(master_edit_distance_dict))

  print('sample ', sample_number, 'finished successfully \n');



Streaming output truncated to the last 5000 lines.
[Parallel(n_jobs=50)]: Done 44907 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44908 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44909 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44910 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44911 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44912 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44913 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44914 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44915 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44916 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44917 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44918 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44919 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44920 tasks      | elapsed:  3.0min
[Parallel(n_jobs=50)]: Done 44921 tasks      | elapse

# Individual Analysis

In [ ]:
#Monte Carlo 0 from guided sample had less than perfect precision.  Find out why.

my_id = 0;

print(sample_path);
with open(sample_path, 'rb') as f:
  sample = pickle.load(f);

my_sample = sample.getSamplebySampleID(my_id);
my_lsh_ids = sample.getLocalSampleIds(my_sample);

out_dict = {};
for i in len(my_lsh_ids):
  cur_id = my_lsh_ids[i];
  cur_sample = sample.getSamplebySampleID(cur_id);
  dist = editdistance.eval(my_sample, cur_sample);

  tup = (cur_id, dist);
  out_dict.append(tup);

print(out_dict);

with open(out_dict, 'w') as convert_file:
     convert_file.write(json.dumps(out_dict))



/content/drive/My Drive/colab/guidedSampling/sample_guided_1.pkl


EOFError: Ran out of input

*italicized text*
# Code Analysis

In [ ]:
'''

## Generate and save Sample
startime = time.time();

print ('starting ', choice_name, ' random sample\n ')
print('Sample parameters:  length_of_dna_sequence:  ', length_of_dna_sequence, '\n');
print('num_of_dna_samples:  ', num_of_dna_samples, '\n');
print('shingle_size:  ', shingle_size, '\n');
print('num_dna_seeds:  ', num_dna_seeds, '\n');
print('max_num_of_changes_per_seq:  ', max_num_of_changes_per_seq, '\n');


sample = DnaSequenceDataSampleAndLSHTable(length_of_dna_sequence, num_of_dna_samples, shingle_size, sampling_choice, num_dna_seeds, max_num_of_changes_per_seq);
sample_id_to_sample_one_way_map = sample.getOneWaySampleIdSampleMap();
sample_id_to_sample_two_way_map = sample.getTwoWaySampleIdSampleMap();

print('sample generated \n');
endtime = time.time();
elapsed_secs = endtime - startime;
elapsed_mins = elapsed_secs / 60;
elapsed = str(elapsed_mins) + ' minutes';
'''
'''
length_of_dna_sequence
num_of_dna_samples
shingle_size
'''

# print('Time to generate sample: ', elapsed, '\n');
with open(sample_path, 'rb') as f:
    sample = pickle.load(f);

with open(sampleid_sample_oneway_path, 'rb') as f:
    sample_id_to_sample_one_way_map = pickle.load(f);

with open(sampleid_sample_twoway_path, 'rb') as f:
    sample_id_to_sample_two_way_map = pickle.load(f);



## Generate random queries

output_data = {};

already_chosen_samples = set();

monte_carlo_data = {};

#Separate for loops is far more space effecient because we don't need sample object for the second loop.
#Second loop will be parallelized due to its call to O(N^2) edit distance function

def job_generateQueriesAndPerformLSHSearchInParallel_ForBasicSampling(i):
  num_of_dna_samples = sample.getNumOfDNASamples();
  update_list = [];

  #Generate queries for Basic Random Sampling method
  my_samp_id = random.randint(0, num_of_dna_samples - 1);
  while (my_samp_id in already_chosen_samples):
    rand_samp = random.randint(0, num_of_dna_samples - 1);

  my_sample = sample.getSamplebySampleID(my_samp_id);
  already_chosen_samples.add(my_samp_id);

  my_sample_id = my_samp_id;


  this_query_dna = change_2N_edit_distance_in_DNA_seq(my_sample, max_num_of_changes_per_seq);

  #Perform LSH Search
  matching_dna_list = sample.getLocalSamples(this_query_dna);
  matching_dna_id_list = sample.getLocalSampleIds(this_query_dna);

  #Make output dictionary
  if (my_sample in matching_dna_list):
    foundOrigDna = "YES";
  else:
    foundOrigDna = "NO"


  update_list.append(('starting_sample_id', my_sample_id));
  update_list.append(('query_for_mc', this_query_dna));
  update_list.append(('matching_dna_id_list', matching_dna_id_list));
  update_list.append(('number_of_matches', len(matching_dna_id_list)));
  update_list.append(('found_original_dna', foundOrigDna));

  output_dict = {};
  output_dict.update(update_list);
  return (i, output_dict);

def job_generateQueriesAndPerformLSHSearchInParallel_ForGuidedSampling(i):
  num_dna_seeds = sample.getNumDnaSeeds();
  update_list = [];


  this_query_dna = sample.getSamplebySampleID(i);

  #Perform LSH Search
  matching_dna_id_list = sample.getLocalSampleIds(this_query_dna);

  #Make output dictionary
  if (i in matching_dna_id_list):
    foundOrigDna = "YES";
  else:
    foundOrigDna = "NO"


  update_list.append(('starting_sample_id', i));
  update_list.append(('sampling_choice', sampling_choice));
  update_list.append(('query_for_mc', 0)); # Kept for consistency across program
  update_list.append(('matching_dna_id_list', matching_dna_id_list));
  update_list.append(('number_of_matches', len(matching_dna_id_list)));
  update_list.append(('found_original_dna', foundOrigDna));

  output_dict = {};
  output_dict.update(update_list);
  return (i, output_dict);


if (sampling_choice == 1):

    output = Parallel(n_jobs=num_jobs, prefer='threads')(delayed(job_generateQueriesAndPerformLSHSearchInParallel_ForBasicSampling)(i) for i in range(0,num_monte_carlos));
elif(sampling_choice == 2):
    output = Parallel(n_jobs=num_jobs, prefer='threads')(delayed(job_generateQueriesAndPerformLSHSearchInParallel_ForGuidedSampling)(i) for i in range(0,num_dna_seeds));
else:
  raise Exception("Bad sampling choice passed.  Sampling choice int:  ", sample_choice, " valid values are 1,2.  For 3 use other block");

output_dict = {};
output_dict.update(output);

print(output_dict);

print('queries generationed \n');

#We only need the 1 way sampleid -> sample map from this point on



#Delete sample because it takes a lot of space
del sample;

def job_getEditDistanceForAllLSHMatches(i, my_mc_data):
  out_this_mc = {};

  sampling_choice = my_mc_data['sampling_choice'];
  starting_sample_id = my_mc_data['starting_sample_id'];
  if (sampling_choice == 1):
    query_for_mc = my_mc_data['query_for_mc'];
  else:
    query_for_mc = sample_id_to_sample_one_way_map[starting_sample_id];
  matching_dna_id_list = my_mc_data['matching_dna_id_list'];
  num_matches = my_mc_data['number_of_matches'];

  matchid_editdistance_pairs = [];
  for j in range(0, num_matches):
    cur_match_id = matching_dna_id_list[j];
    cur_match = sample_id_to_sample_one_way_map[cur_match_id];
    cur_edit_distance = editdistance.eval(query_for_mc, cur_match);

    entry = (cur_match_id, cur_edit_distance);
    matchid_editdistance_pairs.append(entry);

  my_mc_out = {};
  my_mc_out['starting_sample_id'] = starting_sample_id;
  my_mc_out['sampling_choice'] = sampling_choice;

  if (sampling_choice == 1):
    my_mc_out['query_for_mc'] = query_for_mc
  else:
    my_mc_out['query_for_mc'] = 0; # Kept for consistency across program

  my_mc_out['number_of_matches'] = num_matches
  my_mc_out['matchid_editdistance_pairs'] = matchid_editdistance_pairs

  return (i , my_mc_out);

# for sampling_choice == 2, num_monte_carls = num_dna_seeds
if (sampling_choice == 1):
  output = Parallel(n_jobs=num_jobs, prefer='threads')(delayed(job_getEditDistanceForAllLSHMatches)(i, monte_carlo_data[i]) for i in range(0, num_monte_carlos));
else:
  output = Parallel(n_jobs=num_jobs, prefer='threads', verbose=60)(delayed(job_getEditDistanceForAllLSHMatches)(i, output_dict[i]) for i in range(0, num_dna_seeds));

my_mc_data_updated = {};

my_mc_data_updated.update(output);


with open(mc_path_pkl, 'wb') as f:
    pickle.dump(my_mc_data_updated, f);

with open(mc_path_txt, 'w') as convert_file:
     convert_file.write(json.dumps(my_mc_data_updated))

print('test 1 finished successfully! \n');

def job_getRecallForAllQueryStrings(i, cur_mc, sample_id_to_sample_one_way_map_in, num_of_dna_samples):
  out_list_for_mc = [];
  starting_sample_id_int = cur_mc['starting_sample_id'];
  sampling_choice = cur_mc['sampling_choice'];

  if (sampling_choice == 1):
    query_for_mc_str = cur_mc['query_for_mc'];
  else:
    query_for_mc_str = sample_id_to_sample_one_way_map_in[starting_sample_id_int];

  num_matches_int = cur_mc['number_of_matches'];
  matched_sample_ids_edit_distance_list_of_tuples = cur_mc['matchid_editdistance_pairs'];

  list_of_sample_id_edit_distance_pairs = computeAllEditDistancesForQuery(query_for_mc_str, sample_id_to_sample_one_way_map_in, num_of_dna_samples, max_edit_distance);

  del sample_id_to_sample_one_way_map_in;
  gc.collect();

  num_lsh_correctly_matched = 0;
  num_lsh_wrongly_matched = 0;

  num_actual_correctly_matched = 0;

  for j in range(len(matched_sample_ids_edit_distance_list_of_tuples)):
    cur_tuple = matched_sample_ids_edit_distance_list_of_tuples[j];
    cur_sample_id = cur_tuple[0];
    cur_edit_distance = cur_tuple[1];

    if (cur_edit_distance <= max_edit_distance):
      num_lsh_correctly_matched = num_lsh_correctly_matched + 1;
    else:
      num_lsh_wrongly_matched = num_lsh_wrongly_matched + 1;

  for k in range(0, len(list_of_sample_id_edit_distance_pairs)):
    cur_pair = list_of_sample_id_edit_distance_pairs[k];
    cur_pair_sample_id = cur_pair[0];
    cur_pair_edit_distance = cur_pair[1];
    if (cur_pair_edit_distance <= max_edit_distance):
      num_actual_correctly_matched = num_actual_correctly_matched + 1;

  precision = num_lsh_correctly_matched / (num_lsh_correctly_matched + num_lsh_wrongly_matched);
  recall = num_lsh_correctly_matched / num_actual_correctly_matched;
  tup1 = ('precision', precision);
  tup2 = ('recall', recall);
  tup3 = ('starting_sample_id', starting_sample_id_int);
  tup4 = ('query_for_mc', query_for_mc_str);
  tup5 = ('num_matches', num_matches_int);
  out_list_for_mc.append(tup1);
  out_list_for_mc.append(tup2);
  out_list_for_mc.append(tup3);
  out_list_for_mc.append(tup4);
  out_list_for_mc.append(tup5);

  recall_dict_out = {};
  recall_dict_out.update(out_list_for_mc);
  return (i, recall_dict_out);


output = Parallel(n_jobs=num_jobs_recall, verbose=60)(delayed(job_getRecallForAllQueryStrings)(i, my_mc_data_updated[i], sample_id_to_sample_one_way_map, num_of_dna_samples) for i in range(0, num_monte_carlos));

output_dict = {};
output_dict.update(output);

with open(precision_recall_file_path_pkl, 'wb') as f:
    pickle.dump(output_dict, f);

with open(precision_recall_file_path_txt, 'w') as convert_file:
     convert_file.write(json.dumps(output_dict))
print('entire thing finished!')


KeyboardInterrupt: 